<a href="https://colab.research.google.com/github/alemlakes/accuracyAssessmentTools-hfff/blob/main/AccuracyAssessmentTools-hfff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accuracy Assessment Tools Notebook

## Purpose
This notebook demonstrates how to run map-accuracy assessments using the `acc_assessment` package, with examples for **GUE**, **MCEM**, **Stehman**, **Olofsson**, and **Congalton**.

It is designed to help you:
- Load map/reference datasets
- Validate basic data quality and map/reference alignment
- Compute and compare accuracy/area estimates across methods
- Understand differences between probabilistic and crisp workflows

## How to use this notebook
1. Run the setup and import sections first.
2. (Optional) Run the test section to confirm the environment is healthy.
3. Run the integrated four-method workflow for the main end-to-end example. Note that if you use your own data, update the input file paths in the integrated workflow and rerun from the shared-input loading step onward.
4. Then run any method-specific example sections below, as needed.

## Expected inputs (integrated example)
- A map probability table with `id`, `strata`, and class-probability columns
- A reference probability table with matching `id` rows
- A strata population table with `strata` and `population` columns

# Installs and Imports

In [ ]:
# One-time setup / refresh for Colab
import os
import subprocess
import sys

repo_url = "https://github.com/alemlakes/accuracyAssessmentTools-hfff.git"
repo_dir = "/content/accuracyAssessmentTools-hfff"

os.makedirs("/content", exist_ok=True)

if not os.path.exists(repo_dir):
    print("Cloning repository...")
    subprocess.run(["git", "clone", repo_url, repo_dir], check=True)
else:
    print("Repository exists; pulling latest changes...")
    subprocess.run(["git", "-C", repo_dir, "pull", "--ff-only"], check=False)

os.chdir(repo_dir)
print("Installing package...")
subprocess.run([sys.executable, "-m", "pip", "install", "-U", "."], check=True)

print("Setup complete:", repo_dir)

In [ ]:
import pandas as pd
from acc_assessment.olofsson import Olofsson
from acc_assessment.congalton import Congalton
from acc_assessment.stehman import Stehman
from acc_assessment.cardille import Cardille
from acc_assessment.gue import GUE
from acc_assessment.mcem import MCEM

# Verify that the code works as expected

In [ ]:
!pytest

# Integrated Four-Method Example (Same Data)

This section uses one shared probabilistic dataset for all four methods.

Workflow:
1. **Sets** three input file paths (map probabilities, reference probabilities, and strata populations).
2. **Loads** the three files and builds shared inputs.
3. **Runs** **GUE** (analytical) and **MCEM** (simulation) directly on those probabilistic inputs.
4. **Converts** those same probabilities to crisp classes using `argmax`.
5. **Runs** **Stehman** and **Olofsson** on the hardened table.

Note: each code cell prints labeled outputs so you can see what object or result is being shown.

In [ ]:
# Set the three input files used by the integrated four-method example
map_file_same = "./tests/map_data_table.csv"
ref_file_same = "./tests/ref_data_table.csv"
strata_file_same = "./tests/strata_population_table.csv"

print("Map file:", map_file_same)
print("Reference file:", ref_file_same)
print("Strata population file:", strata_file_same)

In [ ]:
# Read the three files and derive shared variables for all four approaches
from IPython.display import display, HTML

prob_map_same = pd.read_csv(map_file_same)
prob_ref_same = pd.read_csv(ref_file_same)
strata_population_df = pd.read_csv(strata_file_same)

# Early alignment checks: map and reference tables must match by id
for table_name, table_df in [("map", prob_map_same), ("reference", prob_ref_same)]:
    if "id" not in table_df.columns:
        raise ValueError(f"{table_name} table is missing required column: 'id'")

if len(prob_map_same) != len(prob_ref_same):
    raise ValueError(
        f"Row count mismatch: map has {len(prob_map_same)} rows, "
        f"reference has {len(prob_ref_same)} rows."
    )

map_ids = prob_map_same["id"].reset_index(drop=True)
ref_ids = prob_ref_same["id"].reset_index(drop=True)

if not map_ids.equals(ref_ids):
    mismatch_rows = map_ids.ne(ref_ids)
    first_bad_row = int(mismatch_rows.idxmax())
    raise ValueError(
        "ID order mismatch between map and reference tables. "
        f"First mismatch at row position {first_bad_row}: "
        f"map id={map_ids.iloc[first_bad_row]!r}, "
        f"reference id={ref_ids.iloc[first_bad_row]!r}."
    )

strata_population_same = dict(
    zip(strata_population_df["strata"], strata_population_df["population"])
)
class_cols_same = [c for c in prob_map_same.columns if c not in ["strata", "id"]]
target_class_same = class_cols_same[0]
N_same = sum(strata_population_same.values())

print("Map/reference alignment check: PASS (same row count and id order)")
print("Strata population table from strata_file_same")
display(strata_population_df.set_index("strata"))

print("\nPreview: prob_map_same (map class probabilities, first 5 rows)")
display(HTML(prob_map_same.head().to_html(index=False)))

print("\nPreview: prob_ref_same (reference class probabilities, first 5 rows)")
display(HTML(prob_ref_same.head().to_html(index=False)))

print("\nPreview: map and reference probabilities side by side (first 5 rows)")
side_by_side_preview = pd.concat(
    [
        prob_map_same.head().add_prefix("map_"),
        prob_ref_same.head().add_prefix("ref_"),
    ],
    axis=1,
    )
display(HTML(side_by_side_preview.to_html(index=False)))

In [ ]:
# Validate shared probabilistic input tables (row counts + row-level data quality checks)
def probability_table_report(df, table_name, class_cols, id_col="id", tolerance=1e-9):
    print(f"\n=== Data quality report for file: {table_name} ===")
    print(f"Rows: {len(df)}")
    print(f"Class columns: {class_cols}")

    if len(class_cols) == 0:
        print("No class probability columns found. Skipping checks.")
        return

    numeric_probs = df[class_cols].apply(pd.to_numeric, errors="coerce")

    has_missing = numeric_probs.isna().any(axis=1)
    has_negative = (numeric_probs < -tolerance).any(axis=1)
    has_gt_one = (numeric_probs > (1 + tolerance)).any(axis=1)
    row_sum = numeric_probs.sum(axis=1)
    sum_not_one = ~row_sum.between(1 - tolerance, 1 + tolerance)

    max_per_row = numeric_probs.max(axis=1)
    n_tied_max = numeric_probs.eq(max_per_row, axis=0).sum(axis=1)
    max_not_unique = n_tied_max > 1

    tied_max_classes = numeric_probs.apply(
        lambda row: ", ".join(
            [
                col
                for col in class_cols
                if abs(float(row[col]) - float(row.max())) <= tolerance
            ]
        ) if not row.isna().any() else "",
        axis=1,
    )

    if id_col in df.columns:
        duplicated_id = df[id_col].duplicated(keep=False)
        row_id = df[id_col]
    else:
        duplicated_id = pd.Series([False] * len(df), index=df.index)
        row_id = df.index

    issue_mask = (
        has_missing
        | has_negative
        | has_gt_one
        | sum_not_one
        | duplicated_id
        | max_not_unique
    )

    issue_report = pd.DataFrame({
        "row position": df.index,
        "id": row_id,
        "row sum": row_sum.round(6),
        "missing or non-numeric": has_missing,
        "negative probability": has_negative,
        "probability gt 1": has_gt_one,
        "row sum not 1": sum_not_one,
        "max not unique": max_not_unique,
        "tied max classes": tied_max_classes,
        "duplicate ID": duplicated_id,
    })

    if issue_mask.any():
        n_issues = int(issue_mask.sum())
        print(f"Rows with potential issues: {n_issues} of {len(df)}")
        flagged = issue_report.loc[issue_mask].copy()

        bool_cols = [
            "missing_or_non_numeric",
            "negative_probability",
            "probability_gt_1",
            "row_sum_not_1",
            "max_not_unique",
            "duplicate_id",
        ]
        for col in bool_cols:
            flagged[col] = flagged[col].map(lambda value: "⚠" if bool(value) else "")

        flagged["row_sum"] = flagged["row_sum"].map(lambda value: f"{float(value):.3f}")
        flagged["tied_max_classes"] = flagged["tied_max_classes"].replace("", "—")

        flagged = flagged.rename(columns={
            "row_position": "Row position",
            "row_sum": "Row sum",
            "missing_or_non_numeric": "Missing/non-numeric",
            "negative_probability": "Negative probability",
            "probability_gt_1": "Probability > 1",
            "row_sum_not_1": "Row sum not 1",
            "max_not_unique": "Max not unique",
            "tied_max_classes": "Tied max classes",
            "duplicate_id": "Duplicate ID",
        })

        if id_col in df.columns:
            flagged = flagged.set_index("id")
            flagged.index.name = "ID"
            display(flagged)
        else:
            flagged = flagged.set_index("Row position")
            display(flagged)
    else:
        print("No row-level issues found under current checks.")

# Shared class columns already defined in previous cell as class_cols_same
probability_table_report(prob_map_same, "tests/map_data_table.csv", class_cols_same)
probability_table_report(prob_ref_same, "tests/ref_data_table.csv", class_cols_same)

### How to read the next outputs

- The next cell creates two assessment objects from the **same probabilistic inputs**: `gue_same` and `mcem_same`.
- Printing an object shows a summary of key metrics for that method.
- GUE reports analytical estimates (with standard errors).
- MCEM reports simulation-based estimates (with percentile intervals).

You can compare their outputs directly because they are using the same input tables and strata totals.

In [ ]:
# Fuzzy / probabilistic assessments on the same shared inputs
gue_same = GUE(
    map_data=prob_map_same,
    ref_data=prob_ref_same,
    strata_col="strata",
    id_col="id",
    strata_population=strata_population_same,
)

mcem_same = MCEM(
    map_data=prob_map_same,
    ref_data=prob_ref_same,
    strata_col="strata",
    id_col="id",
    strata_population=strata_population_same,
    n_simulations=3000,
    random_state=42,
)

# Show both constructed assessment objects explicitly (labeled)
print("Constructed object: gue_same")
display(gue_same)

print("\nConstructed object: mcem_same")
display(mcem_same)

In [ ]:
# Compare key metrics from GUE and MCEM on the same probabilistic data (formatted output)
def _format_point_se(result, decimals=4, zero_tol=1e-12):
    value, uncertainty = result
    value = float(value)
    if isinstance(uncertainty, tuple) and len(uncertainty) == 2:
        lo, hi = float(uncertainty[0]), float(uncertainty[1])
        return f"{value:.{decimals}f} (95% CI: {lo:.{decimals}f}, {hi:.{decimals}f})"
    se = float(uncertainty)
    if abs(se) < zero_tol:
        se = 0.0
    return f"{value:.{decimals}f} ± {se:.{decimals}f}"

print(
    "Target class:",
    target_class_same,
    "(the first class column detected in the map/reference probability files)",
)

print("\n=== GUE (Analytical on probabilistic inputs) ===")
print("Overall Accuracy:", _format_point_se(gue_same.overall_accuracy()))
print(
    f"Reference Area ({target_class_same}):",
    _format_point_se(gue_same.area(target_class_same, reference=True), decimals=2),
)

print("\n=== MCEM (Simulation on probabilistic inputs) ===")
print("Overall Accuracy:", _format_point_se(mcem_same.overall_accuracy()))
print(
    f"Reference Area ({target_class_same}):",
    _format_point_se(mcem_same.area(target_class_same, reference=True), decimals=2),
)

print("\nTip: GUE uses analytical SEs; MCEM uses simulation percentile intervals.")

In [ ]:
# Convert the SAME probabilistic inputs to crisp labels (argmax per row)
map_crisp_labels = prob_map_same[class_cols_same].idxmax(axis=1)
ref_crisp_labels = prob_ref_same[class_cols_same].idxmax(axis=1)

# Build one hardened table used by both crisp methods
crisp_same = pd.DataFrame({
    "id": prob_map_same["id"],
    "Map class": map_crisp_labels,
    "Reference class": ref_crisp_labels,
})

# Build one shared class-population dictionary from hardened map frequencies
mapped_props_same = crisp_same["Map class"].value_counts(normalize=True)
mapped_population_same = {k: float(v * N_same) for k, v in mapped_props_same.items()}

# For a direct comparison, configure Stehman with map class as strata
crisp_same_stehman = crisp_same.copy()
crisp_same_stehman["strata"] = crisp_same_stehman["Map class"]

# Construct both crisp assessment objects from the same hardened data assumptions
stehman_same = Stehman(
    data=crisp_same_stehman,
    strata_col="strata",
    map_col="Map class",
    ref_col="Reference class",
    strata_population=mapped_population_same,
)

olofsson_same = Olofsson(
    crisp_same,
    mapped_population_same,
    map_col="Map class",
    ref_col="Reference class",
)

print("Hardened table used for both crisp methods (head, indexed by id):")
display(crisp_same.set_index("id").head())

In [ ]:
# Crisp assessments from the same hardened table and shared class populations
print("=== STEHMAN (Crisp, shared assumptions) ===")
print("Overall Accuracy:", stehman_same.overall_accuracy())
print(f"Reference Area({target_class_same}):", stehman_same.area(target_class_same, reference=True))

print("\n=== OLOFSSON (Crisp, shared assumptions) ===")
print("Overall Accuracy:", olofsson_same.overall_accuracy())
print(f"Reference Area({target_class_same}):", olofsson_same.area(target_class_same))

print("\nTip: if assumptions match, point estimates should be close; uncertainty terms may still differ by method.")

# Stehman Assessment Example

The Stehman assessment is based on:
Stehman, S.V., 2014. "Estimating area and map accuracy for stratified
random sampling when the strata are different from the map classes",
International Journal of Remote Sensing. Vol. 35 (No. 13).
https://doi.org/10.1080/01431161.2014.930207

## Load Data

Use the data from Table 2 in Stehman 2014 as the example assessment. Each row represents one pixel. The table must have at least three columns: one for the stratum each pixel was sampled from, one for the map class of each pixel and one for the reference class of each pixel.

In [ ]:
stehman_df = pd.read_csv("./tests/stehman2014_table2.csv", skiprows=1)

In addition to the table of reference data, the Stehman assessment also needs a dictionary containing the total size of each of the strata. Keys in the dictionary should match the labels used in the strata column of the table.

In [ ]:
stehman_strata_populations= {1: 40000, 2: 30000, 3: 20000, 4: 10000}

## Create the Assessment

In [ ]:
stehman_assessment = Stehman(
    data=stehman_df,
    strata_col="Stratum",
    map_col="Map class",
    ref_col="Reference class",
    strata_population=stehman_strata_populations
)
stehman_assessment

The Stehman assessment object has the same `user_accuracy`, `producers_accuracy` methods as the Cardille assessment.

# Olofsson Assessment Example

The Olofsson assessment is based on: Olofsson, P., et al., 2014 "Good practices for estimating area and assessing accuracy of land change", Remote Sensing of Environment. Vol 148 pp. 42-57 https://doi.org/10.1016/j.rse.2014.02.015

## Load Data

Use the data from Table 8 in Olofsson et al. 2014 for the example.

The Olofsson assessment can either be initialized with an error matrix of pixel counts plus a dictionary of the mapped areas or with a longform table of each pixels map and reference values. Both produce the same results, pick the one that matches the form that your data is in.

In [ ]:
olofsson_mapped_populations = {
    "Deforestation": 200000,
    "Forest gain": 150000,
    "Stable forest": 3200000,
    "Stable non-forest": 6450000
}

In [ ]:
# from an error matrix
olofsson_data = [
    [66, 0, 1, 2],
    [0, 55, 0, 1],
    [5, 8, 153, 9],
    [4, 12, 11, 313],
]
classes = ["Deforestation", "Forest gain", "Stable forest", "Stable non-forest"]
olofsson_error_matrix = pd.DataFrame(olofsson_data, index=classes, columns=classes)

olofsson_error_matrix

In [ ]:
olofsson_assessment1 = Olofsson(olofsson_error_matrix, olofsson_mapped_populations)
olofsson_assessment1

In [ ]:
# from a lonfrom table
# first convert the error matrix to the long form
from acc_assessment.utils import _expand_error_matrix
olofsson_longform = _expand_error_matrix(olofsson_error_matrix, "map", "ref")
olofsson_longform

In [ ]:
# to tell it that you are passing a longform table tell it the names of the map
# and the reference columns
olofsson_assessment2 = Olofsson(
    olofsson_longform,
    olofsson_mapped_populations,
    map_col="map",
    ref_col="ref",
)
olofsson_assessment2

## Create the Assessment

# GUE Assessment Example

Use probabilistic map and reference tables to compute analytical estimates with GUE.

In [ ]:
map_table = pd.read_csv("./tests/map_data_table.csv")
ref_table = pd.read_csv("./tests/ref_data_table.csv")
strata_population_dict = {
    "a": 5000,
    "f": 10000,
    "w": 1000,
}

gue_assessment = GUE(
    map_data=map_table,
    ref_data=ref_table,
    strata_col="strata",
    id_col="id",
    strata_population=strata_population_dict,
)
gue_assessment

# MCEM Assessment Example

Run Monte Carlo simulations using the same probabilistic inputs.

In [ ]:
mcem_assessment = MCEM(
    map_data=map_table,
    ref_data=ref_table,
    strata_col="strata",
    id_col="id",
    strata_population=strata_population_dict,
    n_simulations=10000,
)
mcem_assessment

# Comparing GUE to MCEM
The mean of the MCEM simulation distribution should converge to the GUE point estimate. While point estimates (Accuracy and Area) should align, MCEM percentile-based confidence intervals can differ from GUE analytical standard errors, providing a more honest representation of uncertainty for rare classes like deforestation.

In [ ]:
print("=== ANALYTICAL RESULTS (GUE) ===")
print(gue_assessment)

print("\n=== MONTE CARLO RESULTS (MCEM) ===")
print(mcem_assessment)

# Congalton Assessment Example

The Congalton assessment is based on: "A Review of Assessing the Accuracy of Classifications of Remotely Sensed Data", Congalton, R. G., 1991. Remote Sensing of Environment, Vol. 37. pp 35-46 https://doi.org/10.1016/0034-4257(91)90048-B

## Load Data

Use the data from Table 1 in Congalton 1991 to create the example.

In [ ]:
congalton_data = [[65, 4, 22, 24], [6, 81, 5, 8], [0, 11, 85, 19], [4, 7, 3, 90]]
congalton_classes = ["D", "C", "BA", "SB"]
congalton_df = pd.DataFrame(congalton_data, index=congalton_classes, columns=congalton_classes)
congalton_table = pd.DataFrame(_expand_error_matrix(congalton_df, "map", "ref"),)
congalton_table

## Create the Assessment

In [ ]:
congalton_assessment = Congalton(congalton_table, "map", "ref")
congalton_assessment

# Cardille Assessment Example

The Cardille Assessment is based on ongoing work in the Cardille Computational Landscape Ecology Lab.

## Load data

Data should come in two csv files: one containing the map data and one containing the reference data. Each file should have one column containing the point id (to link rows from the map csv file to the reference csv file), and one column for the strata that the point was sampled from, and then one column for each of the possible classes containing the reference/map probability that the point belongs to that class. Column names should match between the two csv files.

In addition to the two csv files you also need to supply a dictionary mapping the stratum to their total size.

In [ ]:
map_table = pd.read_csv("./tests/map_data_table.csv")
map_table

In [ ]:
ref_table = pd.read_csv("./tests/ref_data_table.csv")
ref_table

In [ ]:
strata_population_dict = {
    'a': 5000,
    'f': 10000,
    'w': 1000,
}

## Create the assessment

The Cardille accuracy assessment is a class. An explanation of its constructor can be viewed by calling `help` on the `__init__` function.

In [ ]:
help(Cardille.__init__)

In [ ]:
assessment = Cardille(
    map_data=map_table,
    ref_data=ref_table,
    strata_col="strata",
    id_col="id",
    strata_population=strata_population_dict,
)

## View assessment results

An overview of the assessment can be seen by printing the assessment object.

In [ ]:
assessment

Individual accuracies can be accessed by calling the appropriate method.

These methods all return a tuple of two floats which are the value and the standard error respectively. If the standard error is not calculable it will be returned as `None`.

In [ ]:
forest_users_accuracy = assessment.users_accuracy("Forest")
forest_producers_accuracy = assessment.producers_accuracy("Forest")

print(forest_users_accuracy)
print(forest_producers_accuracy)

To follow the practices outlined in Olofsson 2014 and Stehman 2014, the error matrix is returned as proportion of area by default. You can get an error matrix of point counts by setting `proportions=False`. Note that for the Cardille assessment these counts are scaled by their "weights".

In [ ]:
assessment.error_matrix()

In [ ]:
assessment.error_matrix(proportions=False)

An overview of all the methods can be seen by calling `help`.

In [ ]:
help(assessment)